In [1]:
from selective_bayes import *
import numpy as np
import math
import pandas as pd
from sklearn.metrics import normalized_mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

In [13]:
train = pd.read_csv('train_insurance.csv')
test = pd.read_csv('test_insurance.csv')

In [16]:
sample = pd.read_csv('sample_submission.csv')
test = test.merge(sample)

In [4]:
test['is_claim'].value_counts()

is_claim
1    39063
Name: count, dtype: int64

In [6]:
dataset = pd.concat([train, test], axis=1)
dataset.head()

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0.0,1.0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2.0,1.0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2.0,1.0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,No,No,No,Yes,No,No,No,Yes,0.0,1.0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,No,No,Yes,No,No,No,Yes,0.0,1.0


In [14]:
train['is_claim'].value_counts()

is_claim
0    54844
1     3748
Name: count, dtype: int64

In [17]:
test['is_claim'].value_counts()

is_claim
1    39063
Name: count, dtype: int64

In [7]:
train, test = train_test_split(dataset, test_size=0.1, random_state=9)

In [8]:
train.head()


,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
2393,ID02394,0.888898,0.04,0.432692,C10,73430,1,A,M1,CNG,...,No,Yes,Yes,Yes,No,No,Yes,Yes,5.0,1.0
14693,ID14694,0.139185,0.00,0.336538,C5,34738,1,A,M1,CNG,...,No,No,No,No,No,No,No,Yes,0.0,1.0
554,ID00555,0.592857,0.00,0.605769,C15,290,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2.0,1.0
13269,ID13270,0.211036,0.13,0.538462,C5,34738,3,C2,M4,Diesel,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3.0,1.0
31710,ID31711,0.103268,0.03,0.394231,C2,27003,2,A,M3,Petrol,...,No,No,No,Yes,No,No,No,Yes,0.0,1.0


In [10]:
test.head()

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
56130,ID56131,0.072605,0.02,0.653846,C19,27742,4,B2,M5,Diesel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53284,ID53285,0.613442,0.01,0.461538,C21,3264,1,B2,M6,Petrol,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37173,ID37174,0.317354,0.05,0.471154,C7,6112,1,A,M1,CNG,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2.0,1.0
7464,ID07465,0.085702,0.09,0.432692,C15,290,3,C2,M4,Diesel,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3.0,1.0
30842,ID30843,0.259440,0.09,0.413462,C15,290,1,A,M1,CNG,...,No,Yes,Yes,Yes,No,No,Yes,Yes,5.0,1.0


In [27]:
train = train.drop('policy_id', axis=1)
test = test.drop('policy_id', axis=1)


In [28]:
columns_to_bin = ['population_density', 'policy_tenure', 'age_of_car', 'age_of_policyholder']
#'age_of_car', 'age_of_policyholder']

In [29]:
test['is_claim']

0        1
1        1
2        1
3        1
4        1
        ..
39058    1
39059    1
39060    1
39061    1
39062    1
Name: is_claim, Length: 39063, dtype: int64

In [30]:
train, _ = preprocess_dataset(train, 'is_claim', columns_to_bin=columns_to_bin, columns_to_encode=['is_claim'], duplicates_drop_cols=['population_density', 'policy_tenure', 'age_of_car', 'age_of_policyholder'])
test, _ = preprocess_dataset(test, 'is_claim', columns_to_bin=columns_to_bin, columns_to_encode=['is_claim'], duplicates_drop_cols=['population_density', 'policy_tenure', 'age_of_car', 'age_of_policyholder'])

population_density
policy_tenure
age_of_car
age_of_policyholder
is_claim
population_density
policy_tenure
age_of_car
age_of_policyholder
is_claim


In [32]:
nb = NB_classifier(train, 'is_claim')
nb_preds = []
gold_preds = []
for ind, example in test.iterrows():
    prediction = nb.classify(example)
    nb_preds.append(prediction)
    gold_preds.append(example['is_claim'])
print(f'Accuracy Naive Bayes: {accuracy_score(gold_preds, nb_preds)}')

{0: 6.071834774758446e-13, 1: 1.869537234105357e-14}
{0: 1.221872856578521e-10, 1: 3.2352538926277155e-11}
{0: 1.9118592889170005e-22, 1: 1.577749744931521e-24}
{0: 1.9694829781763225e-13, 1: 4.687092298198163e-15}
{0: 2.5535141257043427e-13, 1: 2.795625116012527e-15}
{0: 1.581994141808087e-17, 1: 4.63971496414764e-18}
{0: 8.318832901458698e-19, 1: 7.729807277634416e-21}
{0: 2.398347636993296e-11, 1: 3.554160500539435e-12}
{0: 2.8098085238356437e-18, 1: 5.197789015333584e-20}
{0: 9.66285223198077e-19, 1: 8.819135893053279e-20}
{0: 6.610345645352196e-23, 1: 3.7327524490975046e-25}
{0: 6.7990241335808e-16, 1: 5.857889726864727e-17}
{0: 9.75764250192941e-16, 1: 8.456768908261154e-17}
{0: 1.3060137459673189e-11, 1: 3.2620707838358537e-12}
{0: 5.951923509054765e-13, 1: 5.5001253953449726e-15}
{0: 3.065046000460866e-23, 1: 2.193043520362079e-25}
{0: 2.7004183411159346e-15, 1: 3.3837451923687557e-16}
{0: 3.5142569246120976e-15, 1: 3.323781993949912e-16}
{0: 1.1707853163897168e-11, 1: 1.714946

In [ ]:
snb = Selective_NB_classifier(train, target_col='is_claim')

In [ ]:
snb.calculate_mutual_information()
snb.order_by_mi()

In [ ]:
snb.select_best_model()